In [2]:
import torch
import timm
import torch.nn as nn
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms

/home/sahil/anaconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = timm.create_model("resnet50", pretrained=True, exportable=True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [9]:
#img = Image.open("/home/sahil/Documents/Classifiers/sample_dog.jpg")
img = Image.open("/home/sahil/Documents/Classifiers/datasets/Plant_health_dataset/images/Test_3.jpg")
img1 = Image.open("/home/sahil/Documents/Classifiers/datasets/Plant_health_dataset/images/Test_4.jpg")

In [12]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor1 = preprocess(img)
input_tensor2 = preprocess(img1)

images = [input_tensor1, input_tensor2]
x = torch.stack(images)
x.shape

torch.Size([2, 3, 224, 224])

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)

In [6]:
#normalized_img = normalized_img.unsqueeze(0)
input_batch.shape

torch.Size([1, 3, 224, 224])

In [13]:
with torch.no_grad():
    output = model(x)

In [18]:
topk = output.topk(5)[1]
topk

tensor([[304, 317, 301, 316,  70],
        [304, 301, 317, 318, 815]])

In [25]:
probabilities = torch.nn.functional.softmax(output, dim=1)
top5_prob, top5_catid = torch.topk(probabilities, 5)
top5_prob, top5_catid

(tensor([[0.3668, 0.2886, 0.0780, 0.0228, 0.0182],
         [0.8516, 0.0384, 0.0117, 0.0114, 0.0099]]),
 tensor([[304, 317, 301, 316,  70],
         [304, 301, 317, 318, 815]]))

In [30]:
y = torch.tensor([304, 64])

In [31]:
correct = 0

In [32]:
y_pred, y_true = torch.argmax(output, axis=1), y.long().squeeze()
correct += (y_pred == y_true).type(torch.float).sum().item()

In [33]:
correct

1.0